### Prologue: Importing packages

In [157]:
import numpy as np
import copy as cp
import scipy, os, time
import random
import params_file as pf
from scipy import stats
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import cProfile #This is to benchmark the code. Recommended by Djole.

In [3]:
from concurrent.futures import ProcessPoolExecutor # for using multiple cores.

## Chapter 1a: Class-related stuff (skip if not using classes)

In [64]:
class Organism(object):
    """To access attributes from the Organism class, do smth along the lines of:
    >>> founder_pop.individuals[0].grn"""
    def __init__(self,name,generation,num_genes,prop_unlinked,prop_no_threshold,thresh_boundaries,decay_boundaries,dev_steps,decays,thresholds,start_vect,grn,development,fitness,genome,proteome):
        self.name=name
        self.generation = generation
        self.num_genes = num_genes
        self.prop_unlinked = prop_unlinked
        self.prop_no_threshold = prop_no_threshold
        self.thresh_boundaries = thresh_boundaries
        self.decay_boundaries = decay_boundaries
        self.dev_steps = dev_steps
        self.decays = decays
        self.thresholds = thresholds
        self.start_vect = start_vect
        self.grn = grn
        self.development = development
        self.genes_on = (self.development.sum(axis=0) != 0).astype(int)
        self.fitness = fitness
        self.genome = genome
        self.proteome= proteome        
        

In [65]:
class Population(object):
    def __init__(self,pop_size,parent=None):
        self.pop_size = pop_size
        self.parent = parent
        self.individuals = None
    
    def populate(self):
        self.individuals = producePop(self.pop_size,self.parent)
    
    def remove_dead(self):
        fitnesses = np.array([ x.fitness for x in self.individuals ])
        self.individuals = self.individuals[ fitnesses > 0 ]
        self.pop_size = self.individuals.size

In [35]:
def makeNewOrganism(parent=None):
    if parent: # This section mutates an ancestor.
        thresh_mutation_rate = pf.thresh_mutation_rate
        decay_mutation_rate = pf.decay_mutation_rate
        thresh_decay_mut_bounds = pf.thresh_decay_mut_bounds
        new_link_bounds = pf.new_link_bounds
        link_mutation_bounds = pf.link_mutation_bounds
        generation = parent.generation + 1
        name = parent.name.split("gen")[0] + "gen" + str(generation)
        start_vect = parent.start_vect
        seq_mutation_rate = pf.seq_mutation_rate
        genome,proteome = mutate_genome(parent.genome,parent.proteome,seq_mutation_rate)
        grn,decays,thresholds,development,fitness = master_mutator(np.array(parent.sequences),np.array(parent.start_vect),int(parent.dev_steps),int(parent.num_mutable_values),np.array(parent.grn),np.array(parent.decays),np.array(parent.thresholds),sequences)
        out_org = Organism(name,generation,parent.num_genes,parent.prop_unlinked,parent.prop_no_threshold,parent.thresh_boundaries,parent.decay_boundaries,parent.dev_steps,decays,thresholds,start_vect,grn,development,fitness,sequences)
    else: # This section creates a new organism from scratch
        num_genes = pf.num_genes
        decay_boundaries = pf.decay_boundaries
        prop_no_threshold = pf.prop_no_threshold
        thresh_boundaries = pf.thresh_boundaries
        prop_unlinked = pf.prop_unlinked
        dev_steps = pf.dev_steps
        name = "Lin" + str(int(np.random.random() * 1000000)) + "gen0"
        decays = randomMaskedVector(num_genes,0,decay_boundaries[0],decay_boundaries[1]) #BUG: check why some values are zero. Decays must never be zero ::--:: not sure this is still happening (7/9/2020 - CJR)
        thresholds = randomMaskedVector(num_genes,prop_no_threshold,thresh_boundaries[0],thresh_boundaries[1])
        start_vect = (lambda x: np.array([1]*1+[0]*(x-1)))(num_genes)#makeStartVect(num_genes)
        grn = makeGRN(num_genes,prop_unlinked)
        development = develop(start_vect,grn,decays,thresholds,dev_steps)
        fitness = calcFitness(development)
        if fitness == 0:
            genome,proteome = None,None
        else:
            genome,proteome = makeGenomeandProteome(pf.seq_length,num_genes,dna_codons,trans_aas)
        out_org = Organism(name,0,num_genes,prop_unlinked,prop_no_threshold,thresh_boundaries,decay_boundaries,dev_steps,decays,thresholds,start_vect,grn,development,fitness,genome,proteome)
    return(out_org)

In [13]:
def producePop(pop_size,parent=None):
    popu = np.ndarray((pop_size,),dtype=object)
    if not parent:
        for i in range(pop_size):
            popu[i] = makeNewOrganism()
    else:
        if type(parent) is Organism:
            for i in range(pop_size):
        #print("Making member number",i,"of the new population")
                popu[i] = makeNewOrganism(parent)
        else:
            print("The type of the parent is not correct",type(parent))
    return(popu)

In [12]:
def founderFinder(min_fitness_val=0):
    founder = makeNewOrganism()
    while founder.fitness <= min_fitness_val:
        founder = makeNewOrganism()
    print(f"Founder found with a fitness value of {founder.fitness}!!")
    return(founder)

## Chapter 1b: Some other (static) declarations.

In [3]:
dna_codons=np.array(['ATA', 'ATC', 'ATT', 'ATG', 'ACA', 'ACC', 'ACG', 'ACT', 'AAC',
       'AAT', 'AAA', 'AAG', 'AGC', 'AGT', 'AGA', 'AGG', 'CTA', 'CTC',
       'CTG', 'CTT', 'CCA', 'CCC', 'CCG', 'CCT', 'CAC', 'CAT', 'CAA',
       'CAG', 'CGA', 'CGC', 'CGG', 'CGT', 'GTA', 'GTC', 'GTG', 'GTT',
       'GCA', 'GCC', 'GCG', 'GCT', 'GAC', 'GAT', 'GAA', 'GAG', 'GGA',
       'GGC', 'GGG', 'GGT', 'TCA', 'TCC', 'TCG', 'TCT', 'TTC', 'TTT',
       'TTA', 'TTG', 'TAC', 'TAT', 'TGC', 'TGT', 'TGG','TAA','TAG','TGA'], dtype=object)

In [4]:
trans_aas=np.array(['I', 'I', 'I', 'M', 'T', 'T', 'T', 'T', 'N', 'N', 'K', 'K', 'S',
       'S', 'R', 'R', 'L', 'L', 'L', 'L', 'P', 'P', 'P', 'P', 'H', 'H',
       'Q', 'Q', 'R', 'R', 'R', 'R', 'V', 'V', 'V', 'V', 'A', 'A', 'A',
       'A', 'D', 'D', 'E', 'E', 'G', 'G', 'G', 'G', 'S', 'S', 'S', 'S',
       'F', 'F', 'L', 'L', 'Y', 'Y', 'C', 'C', 'W','_','_','_'], dtype=object)

In [5]:
data_fields=np.array(["generation","genome","proteome","grn","thresholds","decays","start_vect","development","genes_on","fitness"])

# Chapter 2. Main functions to create an organism and a population
### Both from scratch, and as a next generation

I'll re-do the functions to create an organism, and now they'll be a row of numpy arrays, with the following contents:
* \[0\] generation number (always '0' for the founder)
* \[1\] the genome
* \[2\] the proteome
* \[3\] the grn
* \[4\] the expression threshold levels
* \[5\] the decay lambdas
* \[6\] the starting vector
* \[7\] the development
* \[8\] the 'genes_on' vector
* \[9\] the fitness

I think the rest of the values, like genes on, number of genes, sequence length, etc. can be left for the parameters file, and/or calculated each time from any of the other.

The output now is a population (i.e. an array where each 'row' is an organism array, and because this is a single organism only, it has only one 'row' - a single first dimension. This will hopefully make sense later)

In [53]:
def founder_miner(min_fitness=0.6):
    fitness=0
    while fitness < min_fitness:
        # Importing values for producing the genomic sequences
        n_generation=0
        n_genes=pf.num_genes
        seq_len=pf.seq_length
        genome,proteome=makeGenomeandProteome(seq_len,n_genes,dna_codons,trans_aas)
        #print(genome)
        # Importing the values for producing all the regulatory information.
        prop_off=pf.prop_unlinked # thresholds and decays will have the converse of this probability as 0s. See blow.
        thresh_boundaries=pf.thresh_boundaries # tuple of 2 values.
        decay_boundaries=pf.decay_boundaries # tuple of 2 values.
        grn=makeGRN(n_genes,prop_off)
        thresholds=randomMaskedVector(n_genes,(1-prop_off),thresh_boundaries[0],thresh_boundaries[1])
        decays=randomMaskedVector(n_genes,(1-prop_off),decay_boundaries[0],decay_boundaries[1])
        # Importing values for the developmental info
        dev_steps=pf.dev_steps
        start_vect=(lambda x: np.array([1]*1+[0]*(x-1)))(n_genes)
        development=develop(start_vect,grn,decays,thresholds,dev_steps)
        genes_on=(development.sum(axis=0) != 0).astype(int)
        #print(f"Current fitness {fitness} is lower than minimum {min_fitness}")
        fitness=calcFitness(development)
        out_arr=np.array([np.array((n_generation,genome,proteome,grn,thresholds,decays,start_vect,development,genes_on,fitness),dtype=object)])
    return(out_arr)

The translate_codon() function well...it takes a codon (in DNA format) and outputs the corresponding amino acid. I've also re-written it in lambda-function format (now deleted), out of curiosity mainly, because changing it in the code I think will result in reduced readability.

In [7]:
def translate_codon(codon):
    idx=np.where(dna_codons == codon)[0][0]
    aminoac=trans_aas[idx]
    return(aminoac)

# Chapter 3
## Support functions for making an organism from scratch

#### GRN RELATED
The makeGRN() function below will create a GRN as a numpy array of random regulatory interactions, with a user-defined proportion of interactions set to zero (the "unlinked" ones).

In [8]:
def makeGRN(numGenes,prop_unlinked):
    grn = randomMaskedVector(numGenes ** 2,prop_unlinked,-2,2)
    grn = grn.reshape(numGenes,numGenes)
    return(grn)

#### SEQUENCE RELATED
**Below _were_ the TWO functions that create the sequence arrays. Now they're numpy string arrays of n genes by m codons.**
Must trace back the function calls and correct any possible bugs. _makeRandomSequence()_, and _makeRandomSequenceArray()_.

In [9]:
def makeGenomeandProteome(seq_length,num_genes,dna_codons=dna_codons,trans_aas=trans_aas):
    if seq_length % 3:
#        print("Sequence length",seq_length,"is not a multiple of 3.")
        seq_length = seq_length - (seq_length % 3)
        num_codons = int(seq_length/3)
#        print("Rounding to", seq_length,"for",num_codons,"codons")
    else:
        num_codons=int(seq_length/3)
    idx_vect=np.array(range(0,len(dna_codons)-3))
    genome_arr=np.empty((num_genes,num_codons),dtype=object)
    proteome_arr=np.empty((num_genes,num_codons),dtype=object)
    for i in range(0,num_genes):
        rand_codon_idx=np.hstack((np.random.choice(idx_vect,(num_codons-1)),np.random.choice((61,62,63),1)))
        #len(rand_codons)
        genome_arr[i]=np.array(dna_codons[rand_codon_idx])
        proteome_arr[i]=np.array(trans_aas[rand_codon_idx])
    return(genome_arr,proteome_arr)

#### OTHER SUPPORTING FUNCTIONS

In [10]:
# Function that creates a vector of a given amount of values (within a given range), in which a certain proportion of the values are masked.
def randomMaskedVector(num_vals,prop_zero=0,min_val=0,max_val=1):
    if min_val > max_val:
        print("Error: minimum value greater than maximum value")
        return
    range_size = max_val - min_val
    if prop_zero == 0:
        rpv = np.array(range_size * np.random.random(num_vals) + min_val)
    else:
        mask = np.random.choice((0,1),num_vals,p=(prop_zero,1-prop_zero))
        rpv = np.array(range_size * np.random.random(num_vals) + min_val)
        rpv = (rpv * mask) + 0
    return(rpv)

# Chapter 4:
## Mutation Functions

### Genome Mutation
Essentially, what genome_mutation() does now is:
* Takes in the genome and proteome that will be mutated
* Takes in an array with 3D coordinates for each mutation, in the format \[gene_number,codon_number_ingene,codon_position\]
* For each mutation:
   1. mutate the correct nucleotide
   2. translate the mutated codon
   3. determine what kind of mutation it was (nonsense, non-syn, syn)
   4. add the gene number and the mutation type into the muttype_vect object
* Once this is done, the function outputs the mutated genome, the mutated proteome, and the array that says what type of mutation happened where (nonsense = 0, nonsyn=1, syn=2).

In [11]:
def mutate_genome(gnome,prome,mut_coords):
    mut_num=mut_coords.shape[0] #get the number of rows in the mutation coordinate array, this is the number of mutations
    muttype_vect=np.ndarray((mut_num,2),dtype=object)
    for i in range(mut_num):
        coordinates=mut_coords[i,:]
        #print(coordinates)
        selected_gene=coordinates[0]
        selected_codon_from_gene=coordinates[1]
        selected_codpos=coordinates[2]
        #print((selected_gene,selected_codon_from_gene),selected_codpos)
        selected_codon=gnome[selected_gene,selected_codon_from_gene]
        prev_aacid=translate_codon(selected_codon)
        mutated_codon=pointMutateCodon(selected_codon,selected_codpos)
        gnome[selected_gene,selected_codon_from_gene]=mutated_codon
        new_aacid=translate_codon(mutated_codon)
        if prev_aacid == new_aacid: #Synonymous mutations are plotted as '2'
            muttype=2
        elif new_aacid == "_": # Nonsense mutations are plotted as '0'
            muttype=0
        else: # Nonsynonymous mutations are plotted as '1'
            muttype=1
        prome[selected_gene,selected_codpos]=new_aacid
        muttype_vect[i]=(selected_gene,muttype)
    out_genome=gnome
    out_proteome=prome
    return(out_genome,out_proteome,muttype_vect)

#### codPos function
This function takes in an array of mutation sites (with reference to the whole genome - i.e., a genome of 5 genes with 500 codons each will result in numbers ranging from 0 to 5\*500*3=7500, and outputs a 3D array that, for each mutation, will give a 3-number coordinate in the format: \[gene_number,codon_number(in gene),codon_position]. This will then be moved into the mutation function, and each mutated base can be accessed through genome[gene_number,codon_number][codon_position]. Also it has the obvious benefit of pinpointing exactly where each mutation is occurring.
In the end, I decided to produce 'keys': arrays of the same size of the genome, that for each base have, respectively, the gene number, the codon number in the gene, and the codon position.

In [12]:
def codPos(muts,num_genes,num_codons):
    #base1=num+1
    out_array=np.ndarray((muts.size,3),dtype=object)
    gene_bps=num_codons*3
    genome_bps=gene_bps*num_genes
    genenum_array=np.ndarray((num_genes,gene_bps),dtype=object)
    for i in range(num_genes):
        genenum_array[i,:]=i
    genenum_array=genenum_array.flatten()
    #print("genenum_array:",genenum_array)
    codpos_array=np.tile([0,1,2],num_codons*num_genes)
    #print("codpos_array:",codpos_array)
    codnum_array=np.ndarray((num_genes,gene_bps),dtype=object)
    for i in range(num_genes):
        codnum_array[i,:]=np.repeat(range(num_codons),3)
    codnum_array=codnum_array.flatten()
    #print("codnum_array:",codnum_array)
    for i in range(muts.size):
        basenum=muts[i]
        mut_val=np.array([genenum_array[basenum],codnum_array[basenum],codpos_array[basenum]])
        out_array[i,:]=mut_val
    return(out_array)
    

In [13]:
def randomMutations(in_genome,mut_rateseq):
    total_bases=in_genome.size*3 #Each value in the genome is a codon, so the whole length (in nucleotides) is the codons times 3
    mutations=np.random.choice((0,1),total_bases,p=(1-mut_rateseq,mut_rateseq))
    m=np.array(np.where(mutations != 0)).flatten()
    if m.size:
        output=m
    else:
        output=False
    return(output)

In [158]:
# Input is an organism array, as produced by the founder_miner() function, and the mutation rate of the nucleotide sequence (i.e. mutation probability per base).
def genomeMutatorWrapper(orgarr,mut_rateseq):
    orgarr=cp.deepcopy(orgarr)
    orgarr=orgarr[0]
    in_gen_num=orgarr[0]
    in_genome=orgarr[1]
    in_proteome=orgarr[2]
    in_grn=orgarr[3]
    in_thresh=orgarr[4]
    in_decs=orgarr[5]
    in_start_vect=orgarr[6]
    in_dev=orgarr[7]
    in_genes_on=(in_dev.sum(axis=0) != 0).astype(int)
    in_fitness=orgarr[9]
    mutations=randomMutations(in_genome,mut_rateseq)
    #print(mutations)
    if np.any(mutations):
        mut_coords=codPos(mutations,in_genome.shape[0],in_genome.shape[1])
        #print(mut_coords)
        out_genome,out_proteome,mutlocs=mutate_genome(in_genome,in_proteome,mut_coords)
        out_grn,out_thresh,out_decs=regulator_mutator(in_grn,in_genes_on,in_decs,in_thresh,mutlocs)
        out_dev=develop(in_start_vect,out_grn,out_decs,out_thresh,pf.dev_steps)
        out_genes_on=(out_dev.sum(axis=0) != 0).astype(int)
        out_fitness=calcFitness(out_dev)
    else:
        out_genome=in_genome
        out_proteome=in_proteome
        out_grn=in_grn
        out_thresh=in_thresh
        out_decs=in_decs
        out_dev=in_dev
        out_genes_on=(out_dev.sum(axis=0) != 0).astype(int)
        out_fitness=in_fitness
    out_gen_num=in_gen_num+1
    out_org=np.array([[out_gen_num,out_genome,out_proteome,out_grn,out_thresh,out_decs,out_genes_on,out_dev,out_genes_on,out_fitness]],dtype=object)
    return(out_org)

In [16]:
def pointMutateCodon(codon,pos_to_mutate):
    bases=("T","C","A","G")
    base=codon[pos_to_mutate]
    change = [x for x in bases if x != base]
    new_base = np.random.choice(change)
    split_codon=np.array(list(codon))
    split_codon[pos_to_mutate]=new_base
    new_codon="".join(split_codon)
    return(new_codon)

### GRN mutation

##### weight_mut():
This function was built for the special case of when all the genes are expressed in the genome, so no *actual* synonymous mutation can be done. In such cases, I'll assume that a synonymous mutation is simply a **very minor** change in the GRN, even in an interaction that exists already. However, I realized that this function can be generalized to do any change, using the arguments to control it. For example, non-synonymous changes could also use the same function, but with a higher scale value and activations can be done by giving as a value the average weight, and adding a small percentage as a scaler.

For this, _weight_mut()_ takes in the value, and gives as a result a random number chosen from a uniform distribution that goes from -value(1/_x_) to value(1/_x_), where x is a scaling factor that by default is 100.

So, a synoymous change in weights would be a random number that will be smaller than a hundredth of the current value. Needless to say, this won't work with links that are 'off' (=0).

In [17]:
def weight_mut(value,scaler=0.01):
    val=abs(value) #Make sure value is positive
    if val == 0:
        '''For values at zero, simply get 1, and then modify it by the scale
        This is for activating thresholds that are 0.'''
        val=scaler/scaler
    scaled_val=val*scaler #scale the value
    newVal=value+np.random.uniform(-scaled_val,scaled_val) #add the scaled portion to the total value to get the final result.
    return(newVal)

In [57]:
def threshs_and_decs_mutator(in_thresh,in_dec,mutarr):
    #print(f"Input thresholds were: {in_thresh}")
    #print(f"Input decays were: {in_dec}")
    #print(f"Input mutarr was:\n{mutarr}")
    the_tuple=(in_thresh,in_dec) # make a tuple in which the threshold array is the first value, and the decays the second.
    # This will allow me to easily choose among them at the time of mutating, see within the for loop.
    num_genes=len(in_thresh) #get the number of genes from the amount of values in the thresholds array
    genes=mutarr[:,0] # get the genes to be mutated from the mutarray's 1st column
    #print(f"The array of genes to be mutated is:\n{genes}")
    for i in np.arange(len(genes)): #go through each gene, and decide randomly whether to make a threshold or a decay mutation in the gene.'''
        tuple_idx=np.random.choice((0,1))
        #print(f"Thresholds = 0, Decays = 1, Random choice was = {tuple_idx}")
        gene_num=genes[i] # extract specific gene number that has to be mutated. This maps to the thresh and dec arrays.
        #print(f"This means that gene {gene_num} will be mutated:\nValue {the_tuple[tuple_idx][gene_num]}")
        new_value=abs(weight_mut(the_tuple[tuple_idx][gene_num]))
        the_tuple[tuple_idx][gene_num]=new_value
        #print(f"...is now {new_value}")
    out_thresh,out_decs=(the_tuple[0],the_tuple[1])
    return(out_thresh,out_decs)

### Regulation mutation function
This function looks quite big because it's in charge of translating the sequence mutations into mutations in any of the regulatory interactions (GRN weights, gene decay rates, gene expression thresholds).
Inputs:
* A parental organism (but could be just the GRN)
* The 'muttype_vect' array
    * Came out of the genome mutator function
        * For each mutation:
            * First col: gene number
            * Second col: type of mutation:
                * 0=nonsense
                * 1=non-synonymous
                * 2=synonymous mutations
                
First off, it decides with a biased coin toss whether any of the mutated genes passed in the muttype_vect array will be thresholds or decays. The coin toss is biased because the thresholds and decay rates represent only a small amount of the regulatory interactions present. Precisely, they represent only 2N/(2N+N^2) of the interactions, where N is the number of genes. In this function, I've algebraically simplified the expression to 2/(2+N). If it ends up choosing any number of mutations to be in the thresholds or the decay rates, it calls the "threshs_and_decs_mutation()" function (declared above), and sends over the chosen mutations there, while removing them from the original muttype_vect, so as not to mutate repeatedly the same genes in the GRN.

Then, it goes through each remaining entry in the muttype_vect and mutates the regulatory link/weight according to the following set of rules:
* If the mutation is nonsense ('0'):
    1. Multiply the column and the row of that gene by 0
* If the gene is ON:
    1. Identify all the nonzero values for that gene
    2. If the mutation is non-synonymous ('1'):
        * mutate a random nonzero value with weight_mut(orig_value,0.5). weight_mut() mutates _orig_value_ by adding or subtracting any amount in between the -_p_ and +_p_, _p_ being in this case 0.5, or __half__ the amount of _orig_value_.
    3. If the mutation is synonymous ('2'):
        * mutate a random nonzero a tiiiiiny little with weight_mut(orig_value,0.001). See? here the mutation will never be more than a thousandth of the value. Tiiiiiiiiiny.
    4. Otherwise return 'None' (in case some botched code sends a value that's neither 0,1,or 2)
* If the gene is OFF:
    1. If the mutation is non-synonymous ('1'):
        * identify all __zero/inactive__ values for that gene
        * turn a random *inactive* value __ON__, choosing the number from the mean expressed value, and randomly choosing the sign
    2. If the mutation is synonymous ('2'):
        * identify all __active__ values for that gene
        * choose one at random, and mutate it with weight_mut(orig_value,0.5)
    3. Otherwise also return 'None'
* Return the modified grn, decay rates vector, and expression thresholds vector.


In [58]:
def regulator_mutator(in_grn,genes_on,in_dec,in_thresh,muttype_vect):
    inactive_links=np.array(list(zip(np.where(in_grn == 0)[0],np.where(in_grn == 0)[1])))
    '''I'm adding here a section that decides if any of the mutations will go to the thresholds or the decays.
    If there are any changes that have to happen in the decays and/or thresholds, we can call their mutation
    function. Otherwise we can keep on going.'''
    in_thr=in_thresh
    num_genes=pf.num_genes
    prop=2/(2+num_genes**2) #proportion of mutable sites that are thresholds OR decays
    hits=np.nonzero(np.random.choice((0,1),len(muttype_vect),p=(1-prop,prop)))[0]
    if hits.size > 0:
        mutsarr=muttype_vect[hits]
        #print(f"Sending mutations:\n{mutsarr} to decays/thresholds")
        out_threshs,out_decs=threshs_and_decs_mutator(in_thresh,in_dec,mutsarr)
        muttype_vect=np.delete(muttype_vect,hits,axis=0)
    else:
        out_threshs,out_decs=in_thresh,in_dec
    for i in muttype_vect:
        gene=i[0]
        mtype=i[1]
        #print(f"Gene {gene} has mutation type {mtype}")
        if mtype != 0: # For all non-KO mutations (i.e. synonymous, and non-synonymous)...
            if genes_on[gene]: # If the gene is ON...
                active_links=np.array(list(zip(np.nonzero(in_grn)[0],np.nonzero(in_grn)[1])))
                #print(f"Gene {gene} is ON ({genes_on[gene]}).\nOverall active links are:\n{active_links}")
                actives_in_gene=np.concatenate((active_links[active_links[:,1] == gene,:],active_links[active_links[:,0] == gene,:]),axis=0) # get the gene's active links
                #print(f"Gene {gene}'s active links are:\n{actives_in_gene}, which have the values:\n{in_grn[actives_in_gene]}")
                #print(f"GRN is:\n{in_grn}")
                if mtype == 1: # And the mutation is non-synonymous...
                    #print(f"Mutation {mtype} is NS")
                    #print(f"range to be used is range({len(actives_in_gene)})")
                    rand_idx=np.random.choice(np.arange(len(actives_in_gene))) # FIXED # get a random index number for mutating a link
                    coordinates=tuple(actives_in_gene[rand_idx,:]) # get the random link's specific coordinates
                    val=in_grn[coordinates] # Extract the value that will be mutated.
                    in_grn[coordinates]=weight_mut(val,0.5) # mutate the value.
                    #print(f"Mutating coordinate {coordinates} of the GRN, currently showing the value {val} to {in_grn[coordinates]}")
                elif mtype == 2: # If gene is ON, and the mutation is synonymous...
                    #print(f"Mutation {mtype} is S")
                    #print(f"range to be used is range({len(actives_in_gene)})")
                    rand_idx=np.random.choice(np.arange(len(actives_in_gene))) # FIXED # Same as above
                    coordinates=tuple(actives_in_gene[rand_idx,:]) # Same as above
                    val=in_grn[coordinates] # Same as above
                    in_grn[coordinates]=weight_mut(val,0.001) # mutate the value by a very small amount.
                    #print(f"Mutating coordinate {coordinates} of the GRN a tiny little only, from {val} to {in_grn[coordinates]}")
                else:
                    #print(f"Gene {gene} is neither on nor off, its state is {genes_on[gene]}")
                    None
            else: # If the gene is OFF...
                #print(f"Gene {gene} is OFF ({genes_on[gene]})")
                if mtype == 1: # And the mutation is non-synonymous
                    #print(f"And gene{gene}'s mutation is NS")
                    inactive_links=np.array(list(zip(np.where(in_grn == 0)[0],np.where(in_grn == 0)[1])))
                    inactives_in_gene=np.concatenate((inactive_links[inactive_links[:,1] == gene,:],inactive_links[inactive_links[:,0] == gene,:]),axis=0)
                    rand_idx=np.random.choice(np.arange(len(inactives_in_gene))) # FIXED # Same as above, but with inactives instead
                    coordinates=tuple(inactives_in_gene[rand_idx,:]) # Same as above above             
                    mean_exp_val=np.mean(np.abs(in_grn[np.nonzero(in_grn)])) # Mean expression amount
                    sign=np.random.choice((-1,1)) # Randomly choose between negative or positive
                    new_val=mean_exp_val*sign
                    #print(f"Flipping inactive value at coordinate {coordinates} on at level {new_val}")
                    in_grn[coordinates]=new_val
                elif mtype == 2: # If gene is OFF, and the mutation is synonymous...
                    active_links=np.array(list(zip(np.nonzero(in_grn)[0],np.nonzero(in_grn)[1])))
                    actives_in_gene=np.concatenate((active_links[active_links[:,1] == gene,:],active_links[active_links[:,0] == gene,:]),axis=0) # get the gene's active links
                    #print(f"range to be used is range({len(actives_in_gene)})")
                    rand_idx=np.random.choice(np.arange(len(actives_in_gene))) # FI}XED # get a random index number for mutating a link
                    coordinates=tuple(actives_in_gene[rand_idx,:]) # get the random link's specific coordinates
                    val=in_grn[coordinates] # Extract the value that will be mutated.
                    #print(f"Mutating coordinate {coordinates} of the GRN, currently showing the value {val}")
                    in_grn[coordinates]=weight_mut(val,0.5) # mutate the value.
                else:
                    None
        else:
            in_grn[gene,:]=0
            in_grn[:,gene]=0
            out_grn=in_grn
            #print(f"Knocking out gene{gene}")
    out_grn=in_grn
    return(out_grn,out_threshs,out_decs)

## Development function
Gets the GRN and simulates the development of it through an iterative matrix dot product.

In [20]:
def develop(start_vect,grn,decays,thresholds,dev_steps):
    start_vect = start_vect
#    print(f"Starting with vector: {start_vect}\n and thresholds {thresholds}")
    geneExpressionProfile = np.ndarray(((pf.dev_steps+1),pf.num_genes))
    geneExpressionProfile[0] = np.array([start_vect])
    #Running the organism's development, and outputting the results
    #in an array called geneExpressionProfile
    invect = start_vect
    counter=1
    for i in range(dev_steps):
#      print(f"Development step {counter}")
        decayed_invect = (lambda x, l: x*np.exp(-l))(invect,decays) # apply decay to all gene qties. previously: exponentialDecay(invect,decays)
#        print(f"Shapes of objects to be fed to matmul:\n{grn.shape}\t{decayed_invect.shape}")
        exp_change = np.matmul(grn,decayed_invect) #calculate the regulatory effect of the decayed values.
#        exp_change = myDotProd(grn,decayed_invect) #check my bootleg dot product function
#        print(f"Output of dot product:\n{exp_change}")
        pre_thresholds = exp_change + decayed_invect # add the decayed amounts to the regulatory effects
#        print(f"Result when added:\n{pre_thresholds}")
        thresholder = (pre_thresholds > thresholds).astype(int) # a vector to rectify the resulting values to their thresholds.
#        print(f"Threshold rectifier vector:\n{thresholder}")
        currV = pre_thresholds * thresholder # rectify with the thresholder vect. This step resulted in the deletion of the 'rectify()' function
 #       print(f"Rectifying with the thresholds gives:\n{currV}")
 #      currV = currV
        geneExpressionProfile[(i+1)] = currV
        invect = currV
        counter=counter+1
    return(geneExpressionProfile)

In [21]:
def myDotProd(grn,expression_vect):
    for i in np.arange(expression_vect.size):
        print(f"Using {expression_vect[i]} for the GRN dot product, index {i}")
        grn[:,i]=grn[:,i]*expression_vect[i]
    out=grn.sum(axis=0)
    return(out)

### Fitness functions
The function that calculates the fitness of an organism based on its development, and the accessory functions that calculate parts of it.

In [22]:
def calcFitness(development):
    min_reproducin = pf.min_reproducin
    is_alive = lastGeneExpressed(development,min_reproducin)
    if is_alive:
        genes_on = propGenesOn(development)
        exp_stab = expressionStability(development)
        sim_to_exp = exponentialSimilarity(development)
        fitness_val = np.mean([genes_on,exp_stab,sim_to_exp])
    else:
        fitness_val = 0
    return(fitness_val)
def lastGeneExpressed(development,min_reproducin):
    dev_steps,num_genes = development.shape
    last_col_bool = development[:,(num_genes - 1)] > min_reproducin
    last_val_last_col = development[dev_steps - 1, (num_genes - 1)]
    if last_col_bool.any() and last_val_last_col > 0:
        return_val = True
    else:
        return_val = False
    return(return_val)
def propGenesOn(development):
    genes_on = development.sum(axis=0) > 0
    return(genes_on.mean())
def expressionStability(development):  # I haven't thought deeply about this.
    row_sums = development.sum(axis=1)# What proportion of the data range is
    stab_val = row_sums.std() / (row_sums.max() - row_sums.min()) # the stdev? Less = better
    return(stab_val)
def exponentialSimilarity(development):
    dev_steps,num_genes = development.shape
    row_means = development.mean(axis=1)
    tot_dev_steps = dev_steps
    fitted_line = scipy.stats.linregress(range(tot_dev_steps),np.log(row_means))
    r_squared = fitted_line.rvalue ** 2
    return(r_squared)

### Chapter 5: Population-making functions.
These functions are meant to create a population from an arbitrary number of organisms. Thus, it will take as an input an array of organism arrays and the final total number of organisms in the population, and the reproductive strategy ('equals' means that each organism will be reproduced in equal amounts to fill up the total number of final individuals, 'fitness_linked' means that their ranking following their fitness value will determine the proportion of their offspring present in the final population, and [instert any other desired strategy here])

In [145]:
# Assumes input is a population (i.e. an array of organism arrays), it should crash with a single organism.
def grow_pop(in_orgs,out_pop_size,strategy):
    num_in_orgs=in_orgs.shape[0]
    tot_offspring=np.floor_divide(out_pop_size,num_in_orgs)
    corr_pop_size=tot_offspring*num_in_orgs
    in_orgs=cp.deepcopy(in_orgs)
    print(f"Making a population out of the {num_in_orgs} organisms given, reproductive strategy is {strategy}.\nEach organism will have {tot_offspring} offspring.")
    if strategy == 'equals':
        orgs_per_org=np.array([tot_offspring])
        print(orgs_per_org[0])
    elif strategy == 'fitness_linked':
        print("Some stuff happens")
    else:
        print("Show error message")
    counter=0
    out_pop=np.ndarray((corr_pop_size,),dtype=object)
    for k in range(num_in_orgs): # taking each input organism and adding the requested offspring to the output population.
        num_offsp=orgs_per_org[k]
        for i in range(num_offsp):
            indiv=genomeMutatorWrapper(in_orgs,pf.seq_mutation_rate)[0]
            out_pop[counter]=indiv
            #print(f"Counter #{counter} has an organism {indiv}")
            counter=counter+1

    return(out_pop)

In [168]:
out_pop

NameError: name 'out_pop' is not defined

In [169]:
org0=founder_miner(0.75)

In [170]:
pop1=grow_pop(org0,1000,'equals')

Making a population out of the 1 organisms given, reproductive strategy is equals.
Each organism will have 1000 offspring.
1000


ValueError: 'a' cannot be empty unless no samples are taken

In [166]:
popsel=cp.deepcopy(pop1[range(10)])

In [167]:
pop2=grow_pop(popsel,1000,'equals')

Making a population out of the 10 organisms given, reproductive strategy is equals.
Each organism will have 100 offspring.
100


IndexError: index 1 is out of bounds for axis 0 with size 1

In [101]:
np.all(F1[0][1] == F1[5][1])

False

### Helper functions: doing multi-core analyses...
These functions are from a course I did with the ACRC at the UoB, on parallelizing python. They're meant to prove a point and show an example, not blow anyone's mind.

In [185]:
def slow_add(nsecs, x, y):
    print(f"Process {os.getpid()} going to sleep for {nsecs} second(s)")
    time.sleep(nsecs)
    
    print(f"Process {os.getpid()} waking up")
    return(x+y)

In [186]:
intime=np.array([3,3,3,3,3,3,3,3])
infirst=np.array([8,7,6,5,4,3,2,1])
inlist=np.arange(8)
inlist

array([0, 1, 2, 3, 4, 5, 6, 7])

In [188]:
if __name__ == "__main__":
    with ProcessPoolExecutor() as pool:
        result = pool.map(slow_add, intime,infirst,inlist)

np.array(list(result))

Process 283 going to sleep for 3 second(s)
Process 286 going to sleep for 3 second(s)
Process 293 going to sleep for 3 second(s)
Process 296 going to sleep for 3 second(s)
Process 279 going to sleep for 3 second(s)
Process 304 going to sleep for 3 second(s)
Process 312 going to sleep for 3 second(s)Process 311 going to sleep for 3 second(s)

Process 283 waking up
Process 286 waking up
Process 293 waking up
Process 296 waking upProcess 279 waking up

Process 304 waking up
Process 311 waking upProcess 312 waking up



array([8, 8, 8, 8, 8, 8, 8, 8])

# Testing Ground

In [134]:
org_arr=np.array([founder_miner(0.7)])

In [34]:
org_arr.shape

(1, 10)

# CHECKPOINT:
### Make the following steps into a function that produces a population
Switched the genomeMutationWrapper() to take in an organism array with fields organized as stated in the "data_fields" array, and now, in order to access the same field for all organisms in the population, it suffices to do:
**[ x[9] for x in F1_pop_array[:] ]**, where '9' is the field you want access to, in this case, the fitness value, and 'F1_pop_array' is the population array.

In [25]:
pop_size=1000
num_genes=pf.num_genes
num_codons=np.floor_divide(pf.seq_length,3)
F1_pop_array=np.ndarray((pop_size,10),dtype=object)

In [26]:
F1_pop_array.shape

(1000, 10)

In [135]:
start_time=time.time()
for i in np.arange(pop_size):
    print(f"Currently spawning number {i}")
    F1_pop_array[i]=genomeMutatorWrapper(cp.deepcopy(org_arr),0.00001)
print(f"Time used for producing 1K individuals was {time.time() - start_time} seconds.")

Currently spawning number 0


IndexError: index 1 is out of bounds for axis 0 with size 1

In [68]:
np.sort(np.array([ x[9] for x in F1_pop_array[:] ]))

array([0.        , 0.        , 0.51630709, 0.6505516 , 0.67754043,
       0.68209205, 0.71489295, 0.71492784, 0.71495163, 0.71502598,
       0.71504933, 0.71521433, 0.71521804, 0.71526611, 0.71529177,
       0.71530806, 0.71533142, 0.71540009, 0.71540284, 0.71541442,
       0.71542924, 0.71545285, 0.7154628 , 0.71546407, 0.71546633,
       0.71548837, 0.7154944 , 0.71550001, 0.71550254, 0.7155066 ,
       0.71550683, 0.71550853, 0.71550939, 0.71551017, 0.7155107 ,
       0.71551104, 0.71551109, 0.71551119, 0.71551121, 0.71551121,
       0.71551121, 0.71551121, 0.71551121, 0.71551121, 0.71551121,
       0.71551121, 0.71551121, 0.71551121, 0.71551121, 0.71551121,
       0.71551121, 0.71551121, 0.71551121, 0.71551121, 0.71551121,
       0.71551121, 0.71551121, 0.71551121, 0.71551121, 0.71551121,
       0.71551121, 0.71551121, 0.71551121, 0.71551121, 0.71551121,
       0.71551121, 0.71551121, 0.71551121, 0.71551121, 0.71551121,
       0.71551121, 0.71551121, 0.71551121, 0.71551121, 0.71551

In [120]:
outarr.shape

(100, 10, 333)

In [128]:
F1_pop_genome=outarr

#### Thresholds and decays _before_ mutation

In [51]:
print(f"Old thresholds are:\n{founder.thresholds}\n Old decays are:\n{founder.decays}")

Old thresholds are:
[0.00000000e+00 1.21633178e+00 9.55697815e-03 0.00000000e+00
 2.28582161e-04 6.20393046e-01 1.28873940e-04 3.22561692e-01
 0.00000000e+00 0.00000000e+00]
 Old decays are:
[0.98946285 1.07377454 0.69599319 0.4982124  1.95308117 1.58926341
 1.89817429 1.22058099 0.25832542 0.6748252 ]


In [214]:
weight_mut(a_tupe[0][4])

-0.00011550790990487794

In [182]:
y

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
       34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66,
       68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98])

In [189]:
np.random.choice((x,y),dtype=object)

TypeError: choice() got an unexpected keyword argument 'dtype'